In [ ]:

# Efficient Frontier GUI with Sector Mapping and Constraints

import pandas as pd
import numpy as np
from datetime import datetime, date
from io import BytesIO
from ipywidgets import (
    FileUpload, Button, VBox, Output, Label, SelectMultiple, HBox,
    Text, FloatSlider, Dropdown
)
from IPython.display import display, clear_output
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
import copy

# Global variables
upload = FileUpload(accept=".xlsx", multiple=False)
output_upload = Output()
output_ui = Output()
df_global = None
selected_assets = []
sector_mapping_widgets = {}
sector_constraint_widgets = {}
sector_mapper = {}

def log_returns(df):
    return np.log(df / df.shift(1)).dropna()

def ewma_covariance_matrix(X, alpha):
    T, D = X.shape
    cov = np.zeros((D, D))
    for tt in range(T):
        x_t = X[tt, :]
        if tt == 0:
            cov = x_t[:, np.newaxis] @ x_t[np.newaxis, :]
        else:
            cov = alpha * cov + (1 - alpha) * x_t[:, np.newaxis] @ x_t[np.newaxis, :]
    return cov

def handle_upload(change):
    global df_global
    output_upload.clear_output()
    with output_upload:
        try:
            file_info = upload.value[0]
            content = BytesIO(file_info["content"])
            df = pd.read_excel(content, sheet_name="PyData", engine="openpyxl")
            df = df.rename(columns=df.iloc[0]).drop(0)
            df["Dates"] = pd.to_datetime(df["Dates"])
            df = df.set_index("Dates")
            df = df.astype(float)
            df = df[df.index > datetime(2017, 1, 1)]
            df_global = df
            print("✅ File loaded. Now select assets.")
            show_asset_selector()
        except Exception as e:
            print("❌ Error loading file:", e)

def show_asset_selector():
    asset_select = SelectMultiple(description="Assets:", options=df_global.columns.tolist(), rows=10)
    continue_button = Button(description="➡ Define Sectors", button_style="primary")
    
    def proceed_to_sector_mapping(change):
        global selected_assets
        selected_assets = list(asset_select.value)
        if not selected_assets:
            with output_upload:
                print("⚠️ Please select at least one asset.")
            return
        show_sector_mapping()

    continue_button.on_click(proceed_to_sector_mapping)
    with output_ui:
        clear_output()
        display(VBox([
            Label("📊 Step 2: Select assets to include:"),
            asset_select,
            continue_button
        ]))

def show_sector_mapping():
    global sector_mapping_widgets
    sector_mapping_widgets = {}
    rows = []

    for asset in selected_assets:
        label = Label(value=asset, layout={"width": "200px"})
        dropdown = Dropdown(
            options=["Equities", "Corporate Bonds", "High Yield", "Euro Government Core", "Cash"],
            description="",
            layout={"width": "200px"}
        )
        sector_mapping_widgets[asset] = dropdown
        rows.append(HBox([label, dropdown]))

    next_button = Button(description="➡ Define Sector Constraints", button_style="primary")

    def proceed_to_constraints(change):
        global sector_mapper
        sector_mapper = {asset: dropdown.value for asset, dropdown in sector_mapping_widgets.items()}
        show_sector_constraints()

    next_button.on_click(proceed_to_constraints)

    clear_output(wait=True)
    display(VBox([
        Label("🏷 Step 3: Assign each asset to a sector:"),
        VBox(rows),
        next_button
    ]))

def show_sector_constraints():
    global sector_constraint_widgets
    sector_constraint_widgets = {}
    unique_sectors = set(sector_mapper.values())

    rows = []
    for sector in unique_sectors:
        lower = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description=f"{sector} Min", layout={"width": "45%"})
        upper = FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description=f"{sector} Max", layout={"width": "45%"})
        sector_constraint_widgets[sector] = (lower, upper)
        rows.append(HBox([lower, upper]))

    gamma_slider = FloatSlider(value=0.01, min=0.0, max=0.1, step=0.005, description="L2 Gamma:")
    run_button = Button(description="🚀 Run Optimizer", button_style="success")
    output_result = Output()

    def run_optimizer(change):
        output_result.clear_output()
        with output_result:
            try:
                df = df_global[selected_assets]
                logret = log_returns(df)
                logret = logret.astype(np.float64)
                decay = 0.99
                cov = ewma_covariance_matrix(logret.values, alpha=decay)
                cov = pd.DataFrame(cov * 50, index=df.columns, columns=df.columns)

                mu = pd.Series({asset: 0.05 for asset in selected_assets})  # placeholder default return
                ef = EfficientFrontier(mu, cov, weight_bounds=(0, 1))
                ef.add_objective(objective_functions.L2_reg, gamma=gamma_slider.value)

                # Sector constraints
                sector_lower = {}
                sector_upper = {}
                for sector, (low, up) in sector_constraint_widgets.items():
                    sector_lower[sector] = low.value
                    sector_upper[sector] = up.value

                ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

                min_ret, max_ret = mu.min(), mu.max() - 0.001
                return_range = np.linspace(min_ret, max_ret, 25)
                weights = [ef.efficient_return(r) or ef.weights for r in return_range]
                weights_df = pd.DataFrame(weights, columns=mu.index)
                weights_df["Target Return"] = return_range
                display(weights_df.head())
                print("✅ Optimization completed.")

            except Exception as e:
                print("❌ Error during optimization:", e)

    run_button.on_click(run_optimizer)

    clear_output(wait=True)
    display(VBox([
        Label("📐 Step 4: Set sector constraints:"),
        VBox(rows),
        gamma_slider,
        run_button,
        output_result
    ]))

upload.observe(handle_upload, names="value")

display(VBox([
    Label("📥 Step 1: Upload Excel file (sheet: PyData):"),
    upload,
    output_upload,
    output_ui
]))
